![Descripción de la imagen](images/0.PNG ) 


# Tarea 2. Jbook de PostgresSQL

## 1. Creación de la Tabla `employees`

Este script de Python establece una conexión con una base de datos 'PostgreSQL' para crear la tabla `employees`. Esta tabla almacena información esencial de los empleados, como identificación única, nombre, apellido, correo electrónico, número de teléfono, fecha de contratación, ID del puesto, salario, porcentaje de comisión, y los identificadores del gerente y del departamento.



In [ ]:
import psycopg2

In [ ]:


# Conectar a la base de datos
conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

# Crear la tabla employees
cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    employee_id INTEGER,
    first_name VARCHAR(20),
    last_name VARCHAR(25),
    email VARCHAR(25),
    phone_number VARCHAR(20),
    hire_date DATE,
    job_id VARCHAR(10),
    salary NUMERIC(8,2),
    commission_pct NUMERIC(2,2),
    manager_id INTEGER,
    department_id INTEGER
)
''')

# Crear un índice único para employee_id
cur.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS emp_emp_id_pk
    ON employees (employee_id);
''')

cur.execute('''
ALTER TABLE employees ADD CONSTRAINT employees_pkey PRIMARY KEY (employee_id);
''')

print("Tabla 'employees' creada y configurada exitosamente.")

conn.commit()

cur.close()
conn.close()


### Características de la Tabla

- Se implementa un índice único, `emp_emp_id_pk`, sobre `employee_id` para asegurar la unicidad de cada empleado.
- `employee_id` también se define como la clave primaria, lo que refuerza la integridad de los datos y mejora la eficiencia de las consultas.

![Descripción de la imagen](images/1.PNG "Creacion de Tabla employees") 


### Consultas a `employees`

Realizamos dos consultas para explorar la tabla `employees`:

1. **Vista Previa de Datos**: `SELECT * FROM employees LIMIT 10;` nos da un vistazo a los primeros 10 registros, mostrando un fragmento de los datos de empleados.

2. **Total de Empleados**: `SELECT count(1) FROM employees;` cuenta el número total de empleados, ofreciendo una idea del tamaño total de la tabla.

In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

cur.execute('SELECT * FROM employees LIMIT 10;')

rows = cur.fetchall()

print("Primeras 10 filas de la tabla 'employees':")
for row in rows:
    print(row)

cur.execute('SELECT count(1) FROM employees;')

count = cur.fetchone()[0]
print("\nNúmero total de empleados:", count)

cur.close()
conn.close()

## 2. Creación de la Tabla `courses`

Para organizar y almacenar información sobre los cursos ofrecidos, creamos la tabla `courses` en nuestra base de datos PostgreSQL. Esta tabla incluye detalles clave de cada curso, como su identificador, nombre, autor, estado, y fecha de publicación.

### Esquema de la Tabla

La tabla se compone de las siguientes columnas:

- `course_id`: Un identificador único para cada curso, actuando como clave primaria.
- `course_name`: El nombre del curso, limitado a 60 caracteres.
- `course_author`: El autor del curso, con un máximo de 40 caracteres.
- `course_status`: El estado del curso, que puede ser 'published', 'draft', o 'inactive'.
- `course_published_dt`: La fecha en que el curso fue publicado.


In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

sql = '''
CREATE TABLE IF NOT EXISTS courses (
    course_id SERIAL PRIMARY KEY,
    course_name VARCHAR(60) NOT NULL,
    course_author VARCHAR(40),
    course_status VARCHAR(10) CHECK (course_status IN ('published', 'draft', 'inactive')),
    course_published_dt DATE
)
'''


cur.execute(sql)
print("Tabla 'courses' creada exitosamente.")

conn.commit()
cur.close()
conn.close()


![Descripción de la imagen](images/2.PNG "Creacion de Tabla courses") 

## 3. Inserción de Datos en la Tabla `courses`

Para poblar la tabla `courses` con datos relevantes, ejecutamos un script de Python que inserta una serie de registros representando diferentes cursos ofertados.

### Descripción de la Operación

Insertamos datos variados, incluyendo el nombre del curso, autor, estado (como 'published', 'draft' o 'inactive'), y la fecha de publicación cuando está disponible. Los registros sin fecha de publicación se insertan con un valor nulo para mantener la integridad de la tabla.

### Implementación Técnica

Utilizamos la función `executemany` de psycopg2, lo que permite la inserción eficiente de múltiples registros en una sola operación.


In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

# Datos de la tabla courses para insertar
courses_data = [
    ('Programming using Python', 'Bob Dillon', 'published', '2020-09-30'),
    ('Data Engineering using Python', 'Bob Dillon', 'published', '2020-07-15'),
    ('Data Engineering using Scala', 'Elvis Presley', 'draft', None),
    ('Programming using Scala', 'Elvis Presley', 'published', '2020-05-12'),
    ('Programming using Java', 'Mike Jack', 'inactive', '2020-08-10'),
    ('Web Applications - Python Flask', 'Bob Dillon', 'inactive', '2020-07-20'),
    ('Web Applications - Java Spring', 'Mike Jack', 'draft', None),
    ('Pipeline Orchestration - Python', 'Bob Dillon', 'draft', None),
    ('Streaming Pipelines - Python', 'Bob Dillon', 'published', '2020-10-05'),
    ('Web Applications - Scala Play', 'Elvis Presley', 'inactive', '2020-09-30'),
    ('Web Applications - Python Django', 'Bob Dillon', 'published', '2020-06-23'),
    ('Server Automation - Ansible', 'Uncle Sam', 'published', '2020-07-05')
]

# Insertar datos en la tabla courses
for course in courses_data:
    cur.execute('''
        INSERT INTO courses (course_name, course_author, course_status, course_published_dt)
        VALUES (%s, %s, %s, %s)
    ''', course)

conn.commit()

print("Datos insertados exitosamente en la tabla 'courses'.")

cur.close()
conn.close()


![Descripción de la imagen](images/3.jpg "Creacion de Tabla") 

## 4. Eliminación y Recuento de Cursos en la Tabla `courses`

### Eliminación de Cursos No Activos

Para mantener actualizada la oferta de cursos, hemos eliminado aquellos que no están ni en estado de borrador ('draft') ni publicados ('published') con la siguiente sentencia SQL:

```sql
DELETE FROM courses WHERE course_status NOT IN ('draft', 'published')


### Recuento de Cursos Publicados
Posteriormente, contamos el número de cursos publicados agrupados por autor y los ordenamos de mayor a menor, con la siguiente consulta:

```sql
Copy code
SELECT course_author, COUNT(*) as course_count 
FROM courses 
WHERE course_status = 'published' 
GROUP BY course_author 
ORDER BY course_count DESC

In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

cur.execute("DELETE FROM courses WHERE course_status NOT IN ('draft', 'published')")
conn.commit()

# Obtener y ordenar el recuento de todos los cursos publicados por autor
cur.execute('''
    SELECT course_author, COUNT(*) as course_count 
    FROM courses 
    WHERE course_status = 'published' 
    GROUP BY course_author 
    ORDER BY course_count DESC
''')

published_courses_by_author = cur.fetchall()
for author, count in published_courses_by_author:
    print(f"Author: {author}, Published Courses: {count}")

cur.close()
conn.close()

![Creacion de Tabla](images/4.png) 

## 6. Creación de la Tabla `users`

Se crea la tabla `users` para almacenar información detallada sobre los usuarios. Esta tabla incluye varias columnas que permiten registrar los datos personales y de contacto de los usuarios, así como información de seguimiento como la fecha de nacimiento y el timestamp de creación del registro.

### Estructura de la Tabla

La tabla `users` se define con una variedad de campos:

- `user_id`: Clave primaria autoincremental para identificar de manera única a cada usuario.
- `user_first_name` y `user_last_name`: Para almacenar los nombres de los usuarios.
- `user_email_id`: Dirección de correo electrónico del usuario.
- `user_gender`: Género del usuario, almacenado en un único carácter.
- `user_unique_id`: Identificador único asignado al usuario.
- `user_phone_no`: Número de teléfono del usuario.
- `user_dob`: Fecha de nacimiento del usuario.
- `created_ts`: Marca de tiempo indicando cuándo se creó el registro del usuario.

### SQL para Creación

La tabla se crea mediante la siguiente instrucción SQL:

```sql
CREATE TABLE IF NOT EXISTS users (
    user_id SERIAL PRIMARY KEY,
    user_first_name VARCHAR(30),
    user_last_name VARCHAR(30),
    user_email_id VARCHAR(50),
    user_gender VARCHAR(1),
    user_unique_id VARCHAR(15),
    user_phone_no VARCHAR(20),
    user_dob DATE,
    created_ts TIMESTAMP
)


In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

sql_create_table_users = '''
CREATE TABLE IF NOT EXISTS users (
    user_id SERIAL PRIMARY KEY,
    user_first_name VARCHAR(30),
    user_last_name VARCHAR(30),
    user_email_id VARCHAR(50),
    user_gender VARCHAR(1),
    user_unique_id VARCHAR(15),
    user_phone_no VARCHAR(20),
    user_dob DATE,
    created_ts TIMESTAMP
)
'''
cur.execute(sql_create_table_users)

conn.commit()
print("Tabla 'Users' creada exitosamente.")

cur.close()
conn.close()

### Inserción de Datos en la Tabla `users`
Una vez creada la tabla, procedemos a poblarla con datos iniciales con la ayuda de `psycopg2`. Aquí se incluye una muestra de los registros que se insertan:
| user_first_name | user_last_name | user_email_id             | user_gender | user_unique_id | user_phone_no       | user_dob    | created_ts            |
|-----------------|----------------|---------------------------|-------------|----------------|---------------------|-------------|-----------------------|
| Giuseppe        | Bode           | gbode0@imgur.com          | M           | 88833-8759     | +86 (764) 443-1967  | 1973-05-31  | 2018-04-15 12:13:38   |
| Lexy            | Gisbey         | lgisbey1@mail.ru          | F           | 262501-029     | +86 (751) 160-3742  | 2003-05-31  | 2020-12-29 06:44:09   |
| Karel           | Claringbold    | kclaringbold2@yale.edu    | F           | 391-33-2823    | +62 (445) 471-2682  | 1985-11-28  | 2018-11-19 00:04:08   |
| ...             | ...            | ...                       | ...         | ...            | ...                 | ...         | ...                   |



In [ ]:


# Datos a insertar en la tabla users
users_data = [
    ('Giuseppe', 'Bode', 'gbode0@imgur.com', 'M', '88833-8759', '+86 (764) 443-1967', '1973-05-31', '2018-04-15 12:13:38'),
    ('Lexy', 'Gisbey', 'lgisbey1@mail.ru', 'F', '262501-029', '+86 (751) 160-3742', '2003-05-31', '2020-12-29 06:44:09'),
    ('Karel', 'Claringbold', 'kclaringbold2@yale.edu', 'F', '391-33-2823', '+62 (445) 471-2682', '1985-11-28', '2018-11-19 00:04:08'),
    ('Marv', 'Tanswill', 'mtanswill3@dedecms.com', 'F', '1195413-80', '+62 (497) 736-6802', '1998-05-24', '2018-11-19 16:29:43'),
    ('Gertie', 'Espinoza', 'gespinoza4@nationalgeographic.com', 'M', '471-24-6869', '+249 (687) 506-2960', '1997-10-30', '2020-01-25 21:31:10'),
    ('Saleem', 'Danneil', 'sdanneil5@guardian.co.uk', 'F', '192374-933', '+63 (810) 321-0331', '1992-03-08', '2020-11-07 19:01:14'),
    ('Rickert', "O'Shiels", 'roshiels6@wikispaces.com', 'M', '749-27-47-52', '+86 (184) 759-3933', '1972-11-01', '2018-03-20 10:53:24'),
    ('Cybil', 'Lissimore', 'clissimore7@pinterest.com', 'M', '461-75-4198', '+54 (613) 939-6976', '1978-03-03', '2019-12-09 14:08:30'),
    ('Melita', 'Rimington', 'mrimington8@mozilla.org', 'F', '892-36-676-2', '+48 (322) 829-8638', '1995-12-15', '2018-04-03 04:21:33'),
    ('Benetta', 'Nana', 'bnana9@google.com', 'M', '197-54-1646', '+420 (934) 611-0020', '1971-12-07', '2018-10-17 21:02:51'),
    ('Gregorius', 'Gullane', 'ggullanea@prnewswire.com', 'F', '232-55-52-58', '+62 (780) 859-1578', '1973-09-18', '2020-01-14 23:38:53'),
    ('Una', 'Glayzer', 'uglayzerb@pinterest.com', 'M', '898-84-336-6', '+380 (840) 437-3981', '1983-05-26', '2019-09-17 03:24:21'),
    ('Jamie', 'Vosper', 'jvosperc@umich.edu', 'M', '247-95-68-44', '+81 (205) 723-1942', '1972-03-18', '2020-07-23 16:39:33'),
    ('Calley', 'Tilson', 'ctilsond@issuu.com', 'F', '415-48-894-3', '+229 (698) 777-4904', '1987-06-12', '2020-06-05 12:10:50'),
    ('Peadar', 'Gregorowicz', 'pgregorowicze@omniture.com', 'M', '403-39-5-869', '+7 (267) 853-3262', '1996-09-21', '2018-05-29 23:51:31'),
    ('Jeanie', 'Webling', 'jweblingf@booking.com', 'F', '399-83-05-03', '+351 (684) 413-0550', '1994-12-27', '2018-02-09 01:31:11'),
    ('Yankee', 'Jelf', 'yjelfg@wufoo.com', 'F', '607-99-0411', '+1 (864) 112-7432', '1988-11-13', '2019-09-16 16:09:12'),
    ('Blair', 'Aumerle', 'baumerleh@toplist.cz', 'F', '430-01-578-5', '+7 (393) 232-1860', '1979-11-09', '2018-10-28 19:25:35'),
    ('Pavlov', 'Steljes', 'psteljesi@macromedia.com', 'F', '571-09-6181', '+598 (877) 881-3236', '1991-06-24', '2020-09-18 05:34:31'),
    ('Darn', 'Hadeke', 'dhadekej@last.fm', 'M', '478-32-02-87', '+370 (347) 110-4270', '1984-09-04', '2018-02-10 12:56:00'),
    ('Wendell', 'Spanton', 'wspantonk@de.vu', 'F', None, '+84 (301) 762-1316', '1973-07-24', '2018-01-30 01:20:11'),
    ('Carlo', 'Yearby', 'cyearbyl@comcast.net', 'F', None, '+55 (288) 623-4067', '1974-11-11', '2018-06-24 03:18:40'),
    ('Sheila', 'Evitts', 'sevittsm@webmd.com', None, '830-40-5287', None, '1977-03-01', '2020-07-20 09:59:41'),
    ('Sianna', 'Lowdham', 'slowdhamn@stanford.edu', None, '778-0845', None, '1985-12-23', '2018-06-29 02:42:49'),
    ('Phylys', 'Aslie', 'paslieo@qq.com', 'M', '368-44-4478', '+86 (765) 152-8654', '1984-03-22', '2019-10-01 01:34:28')
]

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

for user in users_data:
    cur.execute('''
        INSERT INTO users (
            user_first_name, user_last_name, user_email_id, user_gender,
            user_unique_id, user_phone_no, user_dob, created_ts
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ''', user)

conn.commit()

cur.close()
conn.close()

print("Usuarios insertados exitosamente.")


![Creacion de Tabla](images/5.png) 

## 7. Análisis de Usuarios por Año de Creación

Este análisis tiene como objetivo determinar el número de usuarios creados cada año en la plataforma. Se utiliza la columna `created_ts` para extraer el año y se cuenta el número total de usuarios registrados.

### Proceso de Consulta

Se aplica la función `EXTRACT(YEAR FROM created_ts)` para obtener el año de creación del usuario y luego se agrupan los datos por este valor. Utilizamos `COUNT(*)` para calcular el número de usuarios y los alias `created_year` y `user_count` para las columnas resultantes.

```sql
SELECT EXTRACT(YEAR FROM created_ts) AS created_year, COUNT(*) AS user_count
    FROM users
    GROUP BY created_year
    ORDER BY created_year ASC;



In [ ]:

from psycopg2 import sql

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()


query = sql.SQL('''
    SELECT EXTRACT(YEAR FROM created_ts) AS created_year, COUNT(*) AS user_count
    FROM users
    GROUP BY created_year
    ORDER BY created_year ASC;
''')

cur.execute(query)

results = cur.fetchall()

print(f"{'created_year':<12} {'user_count':<11}")
for row in results:
    year = int(row[0])  
    count = int(row[1])  
    print(f"{year:<12} {count:<11}")
cur.close()
conn.close()


![Creacion de Tabla](images/6.png) 

## 8. Análisis de Usuarios Nacidos en Mayo

En este análisis, identificamos a los usuarios que nacieron en el mes de mayo y determinamos el día de la semana en que nacieron. Los datos se extrajeron de la tabla `users` y se formatearon para proporcionar una visión clara de la distribución de los cumpleaños a lo largo del mes.

### Proceso de Consulta

La consulta SQL diseñada extrae el `user_id`, la `user_dob` (fecha de nacimiento), el `user_email_id` y utiliza funciones de fecha para generar el `user_day_of_birth` (día de la semana). Para obtener solo aquellos usuarios que nacieron en mayo, se filtraron los resultados por el mes de la fecha de nacimiento.
```sql
SELECT 
    user_id,
    user_dob,
    user_email_id,
    TO_CHAR(user_dob, 'Day') as user_day_of_birth
FROM 
    users
WHERE 
    EXTRACT(MONTH FROM user_dob) = 5
ORDER BY 
    EXTRACT(DAY FROM user_dob);


In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

cur.execute('''
SELECT 
    user_id,
    user_dob,
    user_email_id,
    TO_CHAR(user_dob, 'Day') as user_day_of_birth
FROM 
    users
WHERE 
    EXTRACT(MONTH FROM user_dob) = 5
ORDER BY 
    EXTRACT(DAY FROM user_dob);
''')

results = cur.fetchall()

print(f"{'user_id':<8} {'user_dob':<15} {'user_email_id':<30} {'user_day_of_birth':<15}")
for row in results:
    user_dob_str = str(row[1]) if row[1] else "" 
    print(f"{row[0]:<8} {user_dob_str:<15} {row[2]:<30} {row[3]:<15}")


cur.close()
conn.close()


![Creacion de Tabla](images/7.png) 

## 9. Usuarios Registrados en 2019

El siguiente análisis muestra los usuarios que se añadieron en el año 2019, junto con su nombre completo, ID de correo electrónico y fecha de creación de la cuenta.

### Proceso de Consulta

La consulta se centra en el año de creación de la cuenta (`created_ts`), extrayendo solo aquellos usuarios cuyo registro fue efectuado en el año 2019. Se presenta el nombre del usuario en mayúsculas, como combinación del nombre y apellido, y se ordena la lista resultante por el nombre de usuario.
```sql
SELECT
    user_id,
    UPPER(user_first_name || ' ' || user_last_name) AS user_name,
    user_email_id,
    created_ts,
    EXTRACT(YEAR FROM created_ts) AS created_year
FROM
    users
WHERE
    EXTRACT(YEAR FROM created_ts) = 2019
ORDER BY
    user_name;




In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

cur.execute('''
SELECT
    user_id,
    UPPER(user_first_name || ' ' || user_last_name) AS user_name,
    user_email_id,
    created_ts,
    EXTRACT(YEAR FROM created_ts) AS created_year
FROM
    users
WHERE
    EXTRACT(YEAR FROM created_ts) = 2019
ORDER BY
    user_name;
''')

results = cur.fetchall()

print(f"{'user_id':<8} {'user_name':<20} {'user_email_id':<30} {'created_ts':<25} {'created_year'}")
for row in results:
    print(f"{row[0]:<8} {row[1]:<20} {row[2]:<30} {str(row[3]):<25} {row[4]}")

cur.close()
conn.close()


![Creacion de Tabla](images/8.png) 

## 10. Análisis de Usuarios por Género

Este análisis proporciona una distribución del número de usuarios según su género. Se contabilizan los registros como 'Male', 'Female', o 'Not Specified' en caso de que el género no esté especificado.

### Proceso de Consulta

La consulta SQL agrupa a los usuarios por género utilizando un `CASE` para manejar los géneros no especificados. Se cuenta el número de usuarios en cada categoría y se ordenan los resultados por el recuento en orden descendente.

```sql
SELECT
    CASE
        WHEN user_gender = 'M' THEN 'Male'
        WHEN user_gender = 'F' THEN 'Female'
        ELSE 'Not Specified'
    END AS user_gender,
    COUNT(*) AS user_count
FROM users
GROUP BY user_gender
ORDER BY user_count DESC;



In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

cur.execute('''
SELECT
    CASE
        WHEN user_gender = 'M' THEN 'Male'
        WHEN user_gender = 'F' THEN 'Female'
        ELSE 'Not Specified'
    END AS user_gender,
    COUNT(*) AS user_count
FROM users
GROUP BY user_gender
ORDER BY user_count DESC;
''')

results = cur.fetchall()

print(f"{'user_gender':<15} {'user_count'}")
for row in results:
    print(f"{row[0]:<15} {row[1]}")

cur.close()
conn.close()


![Creacion de Tabla](images/9.png) 

## 11. Extracción de los Últimos 4 Dígitos de IDs Únicos

El análisis siguiente se enfoca en los identificadores únicos de los usuarios, obteniendo los últimos cuatro dígitos de los mismos, siempre que cumplan con el requisito de tener una longitud adecuada.

### Proceso de Consulta

La consulta implementada realiza las siguientes operaciones:
- Elimina los guiones de los identificadores únicos.
- Verifica la longitud del identificador único resultante.
- Extrae los últimos 4 dígitos si la longitud es mayor o igual a 9 caracteres.
- Maneja los valores nulos y aquellos identificadores que no cumplen con la longitud mínima estableciendo los valores 'Not Specified' o 'Invalid Unique Id', respectivamente.

```sql
SELECT
    user_id,
    user_unique_id,
    CASE
        WHEN char_length(regexp_replace(user_unique_id, '-', '', 'g')) < 9 THEN 'Invalid Unique Id'
        WHEN user_unique_id IS NULL THEN 'Not Specified'
        ELSE right(regexp_replace(user_unique_id, '-', '', 'g'), 4)
    END AS user_unique_id_last4
FROM
    users
ORDER BY
    user_id;





In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

cur.execute('''
SELECT
    user_id,
    user_unique_id,
    CASE
        WHEN char_length(regexp_replace(user_unique_id, '-', '', 'g')) < 9 THEN 'Invalid Unique Id'
        WHEN user_unique_id IS NULL THEN 'Not Specified'
        ELSE right(regexp_replace(user_unique_id, '-', '', 'g'), 4)
    END AS user_unique_id_last4
FROM
    users
ORDER BY
    user_id;
''')


results = cur.fetchall()

print(f"{'user_id':<10} {'user_unique_id':<20} {'user_unique_id_last4'}")
for row in results:
    user_unique_id = row[1] if row[1] is not None else 'None'
    user_unique_id_last4 = row[2] if row[2] is not None else 'None'
    print(f"{row[0]:<10} {user_unique_id:<20} {user_unique_id_last4}")


cur.close()
conn.close()


![Creacion de Tabla](images/10.png) 

## 12. Análisis de Usuarios por Código de País

Este análisis muestra el recuento de usuarios agrupados por su código de país, que se extrae de los números de teléfono registrados.

### Proceso de Consulta

La consulta elimina el signo '+' de los números de teléfono y utiliza la función `SUBSTRING` junto con la función `POSITION` para extraer los códigos de país. Se agrupan los datos por código de país y se cuentan los usuarios para cada código.

```sql
SELECT
    SUBSTRING(user_phone_no FROM 2 FOR POSITION(' ' IN user_phone_no) - 2) AS country_code,
    COUNT(*) AS user_count
FROM
    users
WHERE
    user_phone_no IS NOT NULL
GROUP BY
    SUBSTRING(user_phone_no FROM 2 FOR POSITION(' ' IN user_phone_no) - 2)
ORDER BY
    CAST(SUBSTRING(user_phone_no FROM 2 FOR POSITION(' ' IN user_phone_no) - 2) AS INTEGER);


In [ ]:

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)

cur = conn.cursor()

cur.execute('''
SELECT
    SUBSTRING(user_phone_no FROM 2 FOR POSITION(' ' IN user_phone_no) - 2) AS country_code,
    COUNT(*) AS user_count
FROM
    users
WHERE
    user_phone_no IS NOT NULL
GROUP BY
    SUBSTRING(user_phone_no FROM 2 FOR POSITION(' ' IN user_phone_no) - 2)
ORDER BY
    CAST(SUBSTRING(user_phone_no FROM 2 FOR POSITION(' ' IN user_phone_no) - 2) AS INTEGER);
''')


results = cur.fetchall()


print(f"{'country_code':<15} {'user_count'}")
for row in results:
    print(f"{row[0]:<15} {row[1]}")

cur.close()
conn.close()


![Creacion de Tabla](images/11.png) 

## 5. Importación de Datos de Cardano USD (ADA-USD) y Generación de Gráfico de Velas

### Paso 1: Carga y Preparación de Datos
Se importan los datos de precios de Cardano (ADA-USD) desde un archivo CSV utilizando la biblioteca `pandas`. Los nombres de las columnas del `DataFrame` se ajustan para que coincidan con la estructura de la tabla de destino en la base de datos PostgreSQL.


### Paso 2: Creación de la Tabla y Almacenamiento de Datos
Se establece una conexión con la base de datos PostgreSQL y se crea la tabla `cardano_prices` si no existe previamente. Luego, se insertan los datos del `DataFrame` en la tabla recién creada, registrando cada precio y su fecha correspondiente.

```sql
CREATE TABLE IF NOT EXISTS cardano_prices (
    date DATE,
    open NUMERIC,
    high NUMERIC,
    low NUMERIC,
    close NUMERIC,
    adj_close NUMERIC,
    volume NUMERIC
)



In [ ]:
import pandas as pd
import psycopg2

# Leer los datos desde el archivo CSV
df = pd.read_csv('https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/ADA-USD.csv')


df.columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']

conn = psycopg2.connect(
    dbname='myname_db',
    user='myname_user',
    password='password',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS cardano_prices (
    date DATE,
    open NUMERIC,
    high NUMERIC,
    low NUMERIC,
    close NUMERIC,
    adj_close NUMERIC,
    volume NUMERIC
)
''')
conn.commit()

insert_sql = '''
INSERT INTO cardano_prices (date, open, high, low, close, adj_close, volume)
VALUES (%s, %s, %s, %s, %s, %s, %s)
'''


for row in df.itertuples(index=False, name=None):
    cur.execute(insert_sql, row)

conn.commit()
cur.close()
conn.close()


![Descripción de la imagen](images/12.PNG "Creacion de Tabla employees") 

### Paso 3: Análisis Exploratorio de Datos y Visualización
El análisis exploratorio básico proporciona estadísticas descriptivas sobre los precios de Cardano. Además, se genera un gráfico de velas utilizando la biblioteca `plotly.graph_objects` para visualizar la serie de tiempo de los precios, lo que permite identificar tendencias, volatilidad y patrones de comportamiento del precio de ADA-USD.

In [ ]:
import pandas as pd
import plotly.graph_objects as go


df = pd.read_csv('https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/ADA-USD.csv')
df['date'] = pd.to_datetime(df['Date'])
df.set_index('date', inplace=True)

# EDA básico
print(df.describe())

# Gráfico de velas
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.update_layout(title='Gráfico de Velas para Cardano (ADA-USD)',
                  xaxis_title='Fecha',
                  yaxis_title='Precio en USD',
                  xaxis_rangeslider_visible=False)  # Oculta el rango deslizante bajo el gráfico

fig.show()

![Candlestic](images/13.PNG ) 

### EDA
la serie temporal de precios de Cardano (ADA-USD) muestra una media aproximada de \$0.37 tanto para la apertura como para el cierre, indicando que el precio al inicio y al final del día tiende a ser similar en promedio. La desviación estándar alrededor de \$0.61 para ambas métricas sugiere una alta volatilidad en los precios diarios de Cardano.

Los precios máximos (high) y mínimos (low) promedian alrededor de \$0.39 y \$0.35 respectivamente, con una desviación estándar que sugiere movimientos diarios significativos entre los precios altos y bajos. Esto es evidencia de la naturaleza fluctuante de la criptomoneda, que puede experimentar cambios abruptos en períodos cortos.

El volumen de transacciones promedio es de aproximadamente 1.091 mil millones, con picos que alcanzan hasta 19.142 mil millones, lo que muestra el interés y la liquidez variable del activo en el mercado.

### Analisis del Grafico de Velas

El gráfico muestra una etapa inicial de estabilidad con precios bajos, indicando poco movimiento en el mercado. Posteriormente, aparecen picos con alta volatilidad y grandes diferencias de precio en cortos periodos, sugiriendo respuestas del mercado a eventos significativos. Luego, el precio de Cardano se eleva marcadamente, reflejando un crecimiento exponencial que podría señalar un aumento en su adopción. Sin embargo, tras estos picos, se producen caídas drásticas en el precio, típicas de toma de beneficios en un ambiente especulativo. Finalmente, el precio entra en fases de calma relativa, asentándose en nuevos niveles antes de enfrentar más fluctuaciones.